In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_csv('logs/BeamInfo.csv')
df.columns=['E','timestamp', 'protonCount', 'integratedProtons']
df.drop('E',axis=1,inplace=True)
df.timestamp=pd.to_datetime(df.timestamp)

In [ ]:
#group by one second
x=df.groupby(pd.Grouper(key = 'timestamp', freq='1s')).sum()[['protonCount']]

#drop empty times
x=x[x.protonCount>0]

In [ ]:
#find samples only one second difference (pulses from spill split across two seconds) and add them, dropping the second one
t=x.reset_index().timestamp
dt=t.shift(-1)-t

#groups with one second before next
n=(dt==np.timedelta64(1000000000)).values
#next group
np1=(dt.shift()==np.timedelta64(1000000000)).values

#add next group to first
x[n] +=x[np1].values
#drop next group
x.drop(x[np1].index,inplace=True)

In [ ]:
x.to_csv('logs/BeamInfoSummedToSpills.csv')